In [9]:
# Import necessary standard packages
import tensorflow as tf
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

#import tensorflow_addons as tfa

In [2]:
df = pd.read_csv('fold1_test.csv')
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.8

train = df[msk]
test = df[~msk]

In [3]:
trainlist = train['filename'].tolist()

In [4]:
len(trainlist)

2349

In [5]:
testlist = test['filename'].tolist()

In [6]:
import os
path = os.path.abspath(os.getcwd())

In [7]:
trainfiles = []
for f in os.listdir(path):
    file = 'audio/'+f
    if file in trainlist:
        trainfiles.append(f)

In [8]:
testfiles = []
for f in os.listdir(path):
    file = 'audio/'+f
    if file in testlist:
        testfiles.append(f)

In [10]:
trainfiles = []
for f in os.listdir(path):
    file = 'audio/'+f
    if file in trainlist:
        trainfiles.append(f)

In [11]:
def get_label(filepath):
    '''Receives a path to a video and returns its label
    '''
    scn_dict = {'airport': 0, 'shopping_mall': 1, 'metro_station': 2, 
                'street_pedestrian': 3, 'public_square': 4, 'street_traffic': 5,
                'tram': 6, 'bus': 7, 'metro': 8, 'park': 9}
    
    fileid = Path(filepath).name
    scn_id = fileid.split('-')[0]
    label = scn_dict[scn_id]
    return label

# Get labels
train_labels = [get_label(f) for f in trainfiles]
test_labels = [get_label(f) for f in testfiles]

In [12]:
#source: https://medium.com/@hasithsura/audio-classification-d37a82d6715
import numpy as np
import librosa
def get_melspectrogram_db(file_path, sr=None, n_fft=2048, hop_length=512, n_mels=128, fmin=20, fmax=8300, top_db=80):
    wav,sr = librosa.load(file_path,sr=sr)
    if wav.shape[0]<5*sr:
        wav=np.pad(wav,int(np.ceil((5*sr-wav.shape[0])/2)),mode='reflect')
    else:
        wav=wav[:5*sr]
    spec=librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft,
              hop_length=hop_length,n_mels=n_mels,fmin=fmin,fmax=fmax)
    spec_db=librosa.power_to_db(spec,top_db=top_db)
    return spec_db


In [13]:
def spec_to_image(spec, eps=1e-6):
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    spec_min, spec_max = spec_norm.min(), spec_norm.max()
    spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
    spec_scaled = spec_scaled.astype(np.uint8)
    return spec_scaled

In [21]:
import torchvision
img = spec_to_image(get_melspectrogram_db("C:\\Users\\mdsp\\Downloads\\experiments\\test\\airport-lisbon-1175-45070-c.wav"))
tensor_array = torch.from_numpy(img)
tensor_array = tensor_array[None, :]
tensor_array.shape

torch.Size([1, 128, 431])

In [27]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader

In [46]:
import pandas as pd
import torch
import torchaudio
from torch.utils.data import Dataset
from PIL import Image



class DCaseDataset(Dataset):
    """
    Dataloader for DCase dataset
    Structure of the class is taken from:
    https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/audio_classifier_tutorial.ipynb
    """

    labelind2name = {
        0: "airport",
        1: "bus",
        2: "metro",
        3: "metro_station",
        4: "park",
        5: "public_square",
        6: "shopping_mall",
        7: "street_pedestrian",
        8: "street_traffic",
        9: "tram",
    }
    name2labelind = {
        "airport": 0,
        "bus": 1,
        "metro": 2,
        "metro_station": 3,
        "park": 4,
        "public_square": 5,
        "shopping_mall": 6,
        "street_pedestrian": 7,
        "street_traffic": 8,
        "tram": 9,
    }

    def __init__(self, root_dir, csv_path, indices):
        """

        :param root_dir:
        :param split:
        """
        # Open csv files
        self.root_dir = path

        csvData = pd.read_csv(csv_path, sep="\t")
        csvData["label"] = csvData.apply(lambda x: x["filename"].split('/')[1].split("-")[0], axis=1)
        #metaData = pd.read_csv(meta_path, sep="\t") if meta_path is not None else None

        # In test mode, just get file list
        #if split == "test":
        #    self.file_names = []
        #    for i in range(0, len(csvData)):
        #        self.file_names.append(csvData.iloc[i, 0])
        #    return

        # Lists of file names and labels
        self.labels, self.data = [], []
        
        for i in indices:
            filename = csvData.iloc[i, 0].split('/')[1]
            if (csvData.iloc[i, 0].split('/')[1]) in os.listdir(os.path.join(path, "test")):
                #self.file_names.append(csvData.iloc[i, 0].split('/')[1])
                image = spec_to_image(get_melspectrogram_db(os.path.join(path, "test", csvData.iloc[i, 0].split('/')[1])))
                self.data.append(image[np.newaxis,...])
                label = filename.split("-")[0]
                self.labels.append(self.name2labelind[label])
    def __getitem__(self, index):
        """

        :param index:
        :return:
        """

        # Load data
        #filepath = os.path.join(self.root_dir, 'test',self.file_names[index])
        #sound, sfreq = torchaudio.load(filepath, normalize=True)
        #assert sound.shape[0] == 1, "Expected mono channel"
        #sound = torch.mean(sound, dim=0)
        #assert sfreq == 44100, "Expected sampling rate of 44.1 kHz"

        # Remove last samples if longer than expected
        #if sound.shape[-1] >= 441000:
         #   sound = sound[:441000]

        return (self.data[index], self.labels[index])
    

    def __len__(self):
        return len(self.data)

In [47]:


root_dir = "C:\\Users\\mdsp\\Downloads\\experiments\\test\\"
csv_path = "C:\\Users\\mdsp\\Downloads\\experiments\\fold1_test.csv"

N = len(os.listdir(root_dir))
train_indices, test_indices = train_test_split(np.arange(0,N), test_size=0.33)

train_dataset = DCaseDataset(root_dir, csv_path, train_indices)
test_dataset = DCaseDataset(root_dir, csv_path, test_indices)

In [48]:
# import pandas as pd
# import torch
# import torchaudio
# from torch.utils.data import Dataset


# class DCaseDataset(Dataset):
#     """
#     Dataloader for DCase dataset
#     Structure of the class is taken from:
#     https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/audio_classifier_tutorial.ipynb
#     """
    
#     def __init__(self, path, files, labels):
 
#         self.file_names, self.labels = [], []
#         for i in range(len(files)):
#             self.file_names.append(files[i])
#             self.labels.append(labels[i])


#     def __getitem__(self, index):
#         """

#         :param index:
#         :return:
#         """

#         # Load data
#         filepath = path + self.file_names[index]
#         sound, sfreq = torchaudio.load(filepath, normalize=True)
#         assert sound.shape[0] == 1, "Expected mono channel"
#         sound = torch.mean(sound, dim=0)
#         assert sfreq == 44100, "Expected sampling rate of 44.1 kHz"

#         # Remove last samples if longer than expected
#         if sound.shape[-1] >= 441000:
#             sound = sound[:441000]

#         return (
#                 sound,
#                 self.labels[index],
#                 self.file_names[index]
#             )

#     def __len__(self):
#         return len(self.file_names)
    
    

In [49]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader
from sp4asc.models.cnns import LogMelSpectrogram
from sp4asc.models import get_net
from sp4asc.training import TrainingManager

In [50]:
#N = len(os.listdir(root_dir))
#train_indices, test_indices = train_test_split(np.arange(0,N), test_size=0.33)

#train_dataset = DCaseDataset(root_dir, csv_path, train_indices)
#test_dataset = DCaseDataset(root_dir, csv_path, test_indices)

In [100]:
import torch.nn.functional as F
class SimpleCNN(nn.Module):
  def __init__(self, input_shape, batch_size=16, num_cats=10):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size = 3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(32, 32, kernel_size = 3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
    self.bn3 = nn.BatchNorm2d(64)
    self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
    self.bn4 = nn.BatchNorm2d(64)
    self.conv5 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
    self.bn5 = nn.BatchNorm2d(128)
    self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
    self.bn6 = nn.BatchNorm2d(128)
    self.conv7 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
    self.bn7 = nn.BatchNorm2d(256)
    self.conv8 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
    self.bn8 = nn.BatchNorm2d(256)
    self.dense1 = nn.Linear(256*(((input_shape[1]//2)//2)//2)*(((input_shape[2]//2)//2)//2),500)
    self.dropout = nn.Dropout(0.5)
    self.dense2 = nn.Linear(500, num_cats)
  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(self.bn1(x))
    x = self.conv2(x)
    x = F.relu(self.bn2(x))
    x = F.max_pool2d(x, kernel_size=2) 
    x = self.conv3(x)
    x = F.relu(self.bn3(x))
    x = self.conv4(x)
    x = F.relu(self.bn4(x))
    x = F.max_pool2d(x, kernel_size=2)
    x = self.conv5(x)
    x = F.relu(self.bn5(x))
    x = self.conv6(x)
    x = F.relu(self.bn6(x))
    x = F.max_pool2d(x, kernel_size=2)
    x = self.conv7(x)
    x = F.relu(self.bn7(x))
    x = self.conv8(x)
    x = F.relu(self.bn8(x))
    x = x.view(x.size(0),-1)
    x = F.relu(self.dense1(x))
    x = self.dropout(x)
    x = self.dense2(x)
    return x

In [107]:
from torchvision.models import resnet34
import torch
import torch.nn as nn
import torch.optim as optim
if torch.cuda.is_available():
  device=torch.device('cuda:0')
else:
  device=torch.device('cpu')
net = resnet34(pretrained=True)
net.fc = nn.Linear(512,10)
net.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
net = net.to(device)

In [95]:
device = "cpu"
net = SimpleCNN()
net = net.to(device)
print(net)

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=11264, out_features=10, bias=True)
  (batchnorm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [108]:
config = {
    "batchsize": 32,
    "num_workers": 0,
    "reload": False,
    "net": net,
    "dropout": 0.2,
    "specAugment": [128, 2, 16, 2],
    "lr": 1e-3,
    "eta_min": 1e-5,
    "max_epoch": 100,
    "weight_decay": 1e-5,
    "mixup_alpha": 0.2,
    "out_dir": "C:\\Users\\mdsp\\Downloads\\experiments",
}

In [109]:

loader_train = DataLoader(
        train_dataset,
        batch_size=config["batchsize"],
        shuffle=True,
        pin_memory=True,
        num_workers=config["num_workers"],
        drop_last=True,
    )

loader_test = DataLoader(
    test_dataset,
    batch_size=config["batchsize"],
    shuffle=False,
    pin_memory=True,
    num_workers=config["num_workers"],
    drop_last=False,
    
)

In [110]:
from tqdm import tqdm
learning_rate = config["lr"]
optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate)
num_epochs = 50
loss_fn = nn.CrossEntropyLoss()
train_losses=[]
valid_losses=[]

def setlr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer

def lr_decay(optimizer, epoch):
    if epoch%10==0:
        new_lr = learning_rate / (10**(epoch//10))
        optimizer = setlr(optimizer, new_lr)
        print(f'Changed learning rate to {new_lr}')
    return optimizer


def train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, train_losses, valid_losses, change_lr=None):
  for epoch in tqdm(range(1,epochs+1)):
    model.train()
    batch_losses=[]
    if change_lr:
      optimizer = change_lr(optimizer, epoch)
    for i, data in enumerate(train_loader):
      x, y = data
      optimizer.zero_grad()
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      y_hat = model(x)
      loss = loss_fn(y_hat, y)
      loss.backward()
      batch_losses.append(loss.item())
      optimizer.step()
    train_losses.append(batch_losses)
    print(f'Epoch - {epoch} Train-Loss : {np.mean(train_losses[-1])}')
    model.eval()
    batch_losses=[]
    trace_y = []
    trace_yhat = []
    for i, data in enumerate(valid_loader):
      x, y = data
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      y_hat = model(x)
      loss = loss_fn(y_hat, y)
      trace_y.append(y.cpu().detach().numpy())
      trace_yhat.append(y_hat.cpu().detach().numpy())      
      batch_losses.append(loss.item())
    valid_losses.append(batch_losses)
    trace_y = np.concatenate(trace_y)
    trace_yhat = np.concatenate(trace_yhat)
    accuracy = np.mean(trace_yhat.argmax(axis=1)==trace_y)
    print(f'Epoch - {epoch} Valid-Loss : {np.mean(valid_losses[-1])} Valid-Accuracy : {accuracy}')

In [111]:
train(net, loss_fn, loader_train, loader_test, num_epochs, optimizer, train_losses, valid_losses, lr_decay)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

Epoch - 1 Train-Loss : 1.923421493598393


  2%|█▋                                                                                 | 1/50 [01:00<49:05, 60.12s/it]

Epoch - 1 Valid-Loss : 53.97963409423828 Valid-Accuracy : 0.5736434108527132
Epoch - 2 Train-Loss : 0.8270433800561088


  4%|███▎                                                                               | 2/50 [01:59<47:58, 59.96s/it]

...

Epoch - 48 Valid-Loss : 0.7661629635374994 Valid-Accuracy : 0.751937984496124
Epoch - 49 Train-Loss : 0.07598633744886943


 98%|██████████████████████████████████████████████████████████████████████████████▍ | 49/50 [1:06:43<00:59, 59.97s/it]

Epoch - 49 Valid-Loss : 0.7580497388960794 Valid-Accuracy : 0.751937984496124
Changed learning rate to 1e-08
Epoch - 50 Train-Loss : 0.043643808524523466


100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [1:07:43<00:00, 81.27s/it]

Epoch - 50 Valid-Loss : 0.761581274587661 Valid-Accuracy : 0.751937984496124


In [113]:
torch.save(net.state_dict(),"C:\\Users\\mdsp\\Downloads\\experiments\\model\\checkpoint.pth")